In [110]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets
from ipywidgets import interactive,interact, HBox, Layout,VBox
from IPython.display import display

In [111]:
a = ipywidgets.IntSlider()
b = ipywidgets.IntSlider()
c = ipywidgets.IntSlider()
ui = ipywidgets.HBox([a, b, c])
def f(a, b, c):
    print((a, b, c))

out = ipywidgets.interactive_output(f, {'a': a, 'b': b, 'c': c})

display(ui, out)

Output()

In [112]:
processing_steps = np.array([[10000,100,10],[20000,200,20]])
# result = (processing_steps[:,:2]*processing_steps[:,2][:,None])
# result = np.multiply(processing_steps[:,:2],processing_steps[:,[2]])
result = np.multiply(processing_steps,2)
# x = np.dot([[10000,100,10],[20000,200,20]],1)
# x[:,2]
result

array([[20000,   200,    20],
       [40000,   400,    40]])

In [113]:
diction = {"a":10,"b":20,"c":30}
diction["total"] = sum(diction.values())
diction['x']=30300
diction

{'a': 10, 'b': 20, 'c': 30, 'total': 60, 'x': 30300}

In [114]:
def MaterialProduction(EmbodiedMat1, EmbodiedMat2, massTM, massAM):
    EG_TM = np.multiply(EmbodiedMat1,massTM)
    EG_AM = np.multiply(EmbodiedMat2,massAM)
    return EG_TM, EG_AM

In [115]:
def Manufacturing(processing_steps):
    result = np.multiply(processing_steps[:,:2],processing_steps[:,[2]])
    result = np.sum(result, axis=0)
    return result


In [116]:
# def Transport(distanceAM, distanceTM, methodAM, methodTM):
def Transport():
    # https://www.section.io/engineering-education/using-geopy-to-calculate-the-distance-between-two-points/#:~:text=Geopy%20is%20a%20Python%20library,well%20as%20other%20data%20sources.
    # https://towardsdatascience.com/driving-distance-between-two-or-more-places-in-python-89779d691def
    # https://towardsdatascience.com/calculating-distance-between-two-geolocations-in-python-26ad3afe287b
    return 0

In [117]:
def Use(gain,dif_mass):
    result = np.multiply(gain, dif_mass)
    
    return result

In [118]:
def Disposal():
    return 0

In [119]:
def Calculation(imassTM, fmassTM,embodiedTMe,embodiedTMc, embodiedAMe,embodiedAMc,
                TM11,TM12,TM13,
                TM21,TM22,TM23,
                TM31,TM32,TM33,TMsteps,
                AM11,AM12,AM13,
                AM21,AM22,AM23,
                AM31,AM32,AM33,AMsteps,
                mass_reduction_perc, finishing_perc, support_perc,
                eff_energ_conv,
                CO2equivalent,
                typeUse): 
    
    ############## MATERIAL PRODUCTION
#     print(fmassTM, fmassTM*(1-mass_reduction_perc))
    massAM = fmassTM*(1-mass_reduction_perc)/(1-(support_perc+finishing_perc))
#     print(massAM)
    
    TMmat_prod, AMmat_prod = MaterialProduction([embodiedTMe,embodiedTMc] , [embodiedAMe,embodiedAMc] , imassTM, massAM)
    
    ############# MANUFACTURING
    
    removed_massTM = imassTM-fmassTM
    
    TM_processing = np.array([[TM11,TM12,TM13*removed_massTM],[TM21,TM22,TM23*removed_massTM],[TM31,TM32,TM33*removed_massTM]])
    uTM_processing = TM_processing
    uTM_processing[(TMsteps+1):] = 0
    
    AM_processing = np.array([[AM11,AM12,AM13*massAM],[AM21,AM22,AM23*massAM],[AM31,AM32,AM33*massAM]]) 
    uAM_processing = AM_processing
    uAM_processing[(AMsteps+1):] = 0
    
    TMmanuf = Manufacturing(uTM_processing)
    AMmanuf = Manufacturing(uAM_processing)
    
    ############# TRANSPORT
    
    TMtransport = Transport()
    AMtransport = Transport()
    
    ############# USE
    
    # MJ PER KG OF REDUCED WEIGHT
    energySaving = {'Long-Distance Aircraft': 200*(10**3),'Short-Distance Aircraft': 150*(10**3),
                    'Articulated Truck': 260,'Passenger Car - Diesel': 210, 'Passenger Car - Gasoline': 230}
    # KG PER KG OF REDUCED WEIGHT
    CO2Saving = {'Long-Distance Aircraft': 13.6*(10**3),'Short-Distance Aircraft': 10.2*(10**3),
                    'Articulated Truck': 18.5,'Passenger Car - Diesel': 14.9, 'Passenger Car - Gasoline': 15}
    
    valuesUse = [energySaving[typeUse],CO2Saving[typeUse]]
    reducedWeight = fmassTM*(-mass_reduction_perc) 
    AMuse = Use(valuesUse, reducedWeight)
    
    ############# DISPOSAL
    
    TMdisposal = Disposal()
    AMdisposal = Disposal()
    
    EnergyBarsTM  = {'Material Production':TMmat_prod[0], 'Manufacturing':TMmanuf[0], 'Transport':TMtransport, 'Use':0, 'Disposal':TMdisposal}
#     CO2BarsTM = {'Material Production':30-x, 'Manufacturing':30-y, 'Transport':30-z, 'Use':30-d, 'Disposal':30-d}
    
    EnergyBarsTM["Total"] = sum(EnergyBarsTM.values())
    
    EnergyBarsAM  = {'Material Production':AMmat_prod[0], 'Manufacturing':AMmanuf[0], 'Transport':AMtransport, 'Use':AMuse[0], 'Disposal':AMdisposal}
#     CO2BarsAM = {'Material Production':30-x, 'Manufacturing':30-y, 'Transport':30-z, 'Use':30-d, 'Disposal':30-d}
    EnergyBarsAM["Total"] = sum(EnergyBarsAM.values())
    
    return (EnergyBarsTM,EnergyBarsAM)
#     return (EnergyBarsTM,CO2BarsTM,EnergyBarsAM,CO2BarsAM)

In [125]:
Calculation(imassTM = 5.13, fmassTM=2.04,embodiedTMe=300,embodiedTMc=39.5, embodiedAMe=723,embodiedAMc=40,
              TM11 = 2.28,TM12 = 0.17,TM13 = 0.85,
              TM21 = 2.28,TM22 = 0.17,TM23 = 0.15,
              TM31 = 0,TM32 = 0,TM33 = 0,TMsteps=2,
              AM11 = 176.5,AM12 = 9.6,AM13 = 1,
              AM21 = 18.5,AM22 = 20.4,AM23 = 0.1,
              AM31 = 0,AM32 = 0,AM33 = 0,AMsteps=2,
              mass_reduction_perc=0, finishing_perc=0.1, support_perc = 0.2,
              eff_energ_conv = 0.4,
              CO2equivalent = 20,
              typeUse = 'Long-Distance Aircraft')

({'Material Production': 1539.0,
  'Manufacturing': 7.045199999999999,
  'Transport': 0,
  'Use': 0,
  'Disposal': 0,
  'Total': 1546.0452},
 {'Material Production': 2107.0285714285715,
  'Manufacturing': 519.7628571428572,
  'Transport': 0,
  'Use': 0.0,
  'Disposal': 0,
  'Total': 2626.7914285714287})

In [121]:
def Completed(imassTM = 5.13, fmassTM=2.04,embodiedTMe=668,embodiedTMc=39.5, embodiedAMe=723,embodiedAMc=40,
              TM11 = 2.28,TM12 = 0.17,TM13 = 0.85,
              TM21 = 18.5,TM22 = 1.53,TM23 = 0.15,
              TM31 = 0,TM32 = 0,TM33 = 0,TMsteps=2,
              AM11 = 176.5,AM12 = 9.6,AM13 = 1,
              AM21 = 18.5,AM22 = 1.39,AM23 = 0.1,
              AM31 = 0,AM32 = 0,AM33 = 0,AMsteps=2,
              mass_reduction_perc=0, finishing_perc=0.1, support_perc = 0.2,
              eff_energ_conv = 0.4,
              CO2equivalent = 20,
              typeUse = 'Long-Distance Aircraft', solver = True):
    
    
    EnergyBarsTM, EnergyBarsAM = Calculation(imassTM, fmassTM,embodiedTMe,embodiedTMc, embodiedAMe,embodiedAMc,
                                            TM11,TM12,TM13,
                                            TM21,TM22,TM23,
                                            TM31,TM32,TM33,TMsteps,
                                            AM11,AM12,AM13,
                                            AM21,AM22,AM23,
                                            AM31,AM32,AM33,AMsteps,
                                            mass_reduction_perc, finishing_perc, support_perc,
                                            eff_energ_conv ,
                                            CO2equivalent ,
                                            typeUse)
    
    
    
    
    mass_reduc_solver=0.1
    step = 0.1
    itera = 0
    last_result = 0
    
    while solver and (itera<20):
        EnergyBarsTMs, EnergyBarsAMs = Calculation(imassTM, fmassTM,embodiedTMe,embodiedTMc, embodiedAMe,embodiedAMc,
                                            TM11,TM12,TM13,
                                            TM21,TM22,TM23,
                                            TM31,TM32,TM33,TMsteps,
                                            AM11,AM12,AM13,
                                            AM21,AM22,AM23,
                                            AM31,AM32,AM33,AMsteps,
                                            mass_reduc_solver, finishing_perc, support_perc,
                                            eff_energ_conv,
                                            CO2equivalent ,
                                            typeUse)
        
        itera += 1
        if (EnergyBarsTMs["Total"] - EnergyBarsAMs["Total"] > 0) :
            result = -1 
        else: 
            result = +1
            
        if itera == 1:
            last_result = result
            
        if last_result != result:
            step = step/2
            
        mass_reduc_solver = mass_reduc_solver + result*step
        last_result = result
    
    print("Equilibrium of reduction in mass: ",mass_reduc_solver)
    
    fig, (ax1,ax2) = plt.subplots(2)
    
    names = list(EnergyBarsTM.keys())
    valores = list(EnergyBarsTM.values())
    
    ax1.bar(names,valores)

    names = list(EnergyBarsAM.keys())
    valores = list(EnergyBarsAM.values())
    
    ax2.bar(names,valores)
    
    fig.set_size_inches(18.5, 10.5)
    
    plt.show(fig)
    
    

In [122]:
widget = ipywidgets.interactive(Completed,solver = True, mass_reduction_perc=(0,1,0.02), typeUse = ['Long-Distance Aircraft','Short-Distance Aircraft',
                    'Articulated Truck','Passenger Car - Diesel', 'Passenger Car - Gasoline'])
controls = HBox(widget.children[:-1], layout = Layout(flex_flow='row wrap'))
output = widget.children[-1]
display(VBox([controls, output]))

In [ ]:
fig, (ax1,ax2) = plt.subplots(2)
EnergyBarsTM  = {'Material Production':20, 'Manufacturing':20, 'Transport':20, 'Use':20, 'Disposal':20}
names = list(EnergyBarsTM.keys())
valores = list(EnergyBarsTM.values())

ax1.bar(names,valores)
ax2.bar(names,valores)

In [ ]:
def Complete(x=0,y=0,z=0,d=0, mass_reduction_perc=0.2):
    
    EnergyBars = {'Material Production':30-x, 'Manufacturing':30-y, 'Transport':30-z, 'Use':30-d, 'Disposal':30-d}
    CO2Bars = {'Material Production':30-x, 'Manufacturing':30-y, 'Transport':30-z, 'Use':30-d, 'Disposal':30-d}
    names = list(barras.keys())
    valores = list(barras.values())
    valores.append()
    fig, (ax1,ax2) = plt.subplots
    plt.bar(nomes,valores)

In [ ]:
eff_energ_conv = 0.4 # MJ/MJ %
CO2equivalent = 20 #g/MJ

In [ ]:

##### TRADITIONAL MANUFACTURING
initialMassTM = 100 #kg
finalMassTM = 76 #kg
# Primary Production

embodied_MatTM =[10000,100] # [MJ/kg, kg/kg]

processingTM = [[10000,0.3]] #Cada linha da matriz é um dos passos, primeiro item energia e por ultimo a massa retirada (colocar em % ou em massa mesmo?)





##### ADITIVE MANUFACTURING

embodied_MatAM =[10000,100] # [MJ/kg, kg/kg]

processingAM = [[10000,1.02*finalMass],[1000,0.02]]

